In [5]:
import json
with open('/home/ljh/CSC/Enhanced_Syllable_Feature/data/counter.json', 'r') as f:
    counter = json.load(f)

zi_list = sorted(list(counter.keys()), key= lambda x: counter[x])

for index in range(0,len(zi_list),len(zi_list) // 10):
    print(zi_list[index : index+10])

index = int(len(zi_list) * 0.4)
print(zi_list[index : index+30])

['戓', '炰', '僝', '僽', '忺', '笯', '姱', '胾', '臁', '踧']
['噼', '嘭', '瘼', '疐', '拑', '滆', '厳', '颃', '衒', '囟']
['瘗', '鹡', '鲅', '偎', '晧', '泐', '遛', '垴', '瞑', '赅']
['鸱', '藿', '呦', '拏', '仟', '磴', '胝', '抡', '咭', '蛆']
['砺', '盱', '咱', '牍', '砥', '毎', '轲', '怂', '糠', '恿']
['筿', '鲱', '颉', '佟', '湳', '歙', '璜', '巍', '嶋', '胪']
['洼', '闰', '缮', '诬', '掀', '吟', '洽', '堰', '镍', '桨']
['抛', '惧', '奸', '弄', '抄', '肢', '夸', '呎', '棕', '绥']
['挑', '篮', '倍', '纵', '慈', '莲', '紫', '塘', '邓', '晶']
['拔', '瑞', '照', '征', '望', '述', '依', '预', '注', '督']
['在', '一', '年', '的']
['盱', '咱', '牍', '砥', '毎', '轲', '怂', '糠', '恿', '苌', '浬', '箍', '韭', '鳝', '郗', '褪', '囤', '闫', '帑', '鲭', '馒', '窍', '锆', '嵯', '澹', '猥', '辎', '瓢', '嘘', '锗']


In [12]:

import sys
import opencc
import json
if sys.path[0] != '/home/ljh/CSC/Enhanced_Syllable_Feature':
    sys.path.insert(0,  '/home/ljh/CSC/Enhanced_Syllable_Feature')
from datasets.utils import pho_convertor

converter = opencc.OpenCC('t2s.json')


def traditional_to_simple(text):
    text = converter.convert(text)
    text = text.replace('著', '着').replace('妳', '你')
    return text


def _is_chinese_char(cp):
    if ((cp >= 0x4E00 and cp <= 0x9FFF) or  #
        (cp >= 0x3400 and cp <= 0x4DBF) or  #
        (cp >= 0x20000 and cp <= 0x2A6DF) or  #
        (cp >= 0x2A700 and cp <= 0x2B73F) or  #
        (cp >= 0x2B740 and cp <= 0x2B81F) or  #
        (cp >= 0x2B820 and cp <= 0x2CEAF) or
        (cp >= 0xF900 and cp <= 0xFAFF) or  #
            (cp >= 0x2F800 and cp <= 0x2FA1F)):  #
        return True
    return False


def is_chinese_token(token):
    if len(token) == 1:
        return _is_chinese_char(ord(token))
    # if token.startwith('##') and len(token)==3:
    #     return  _is_chinese_char(ord(token[2]))
    return False
def get_confusionset():
    with open('/home/ljh/github/CHINESE_GRC_DATA/merge_confusion.json', 'r', encoding='utf-8') as f:
        confusionset = json.load(f)
    vocab_file = '/home/ljh/model/ChineseBERT-base/vocab.txt'
    chinese_list = []
    with open(vocab_file, 'r', encoding='utf-8', errors='ignore')as f:
        for line in f.readlines():
            token = line.strip()
            if len(token) == 1 and _is_chinese_char(ord(token)):
                chinese_list.append(converter.convert(token))
    chinese_list = list(set(chinese_list))
    print(len(confusionset))
    for key in confusionset:
        confusionset[key] = [val for val in confusionset[key] if val in chinese_list]
    new_confusion = {}
    for key in confusionset:
        if key in chinese_list:
            new_confusion[key] = confusionset[key]
    print('fix len',len(new_confusion))
    confusionset = new_confusion
    import numpy as np 
    def statistics(num_list):
        print('avg min/max',sum(num_list) / len(num_list), min(num_list), max(num_list))
        sorted_list = sorted(num_list)
        for quantile in np.arange(0,1,0.1):
            print('quantile', quantile, sorted_list[int(len(sorted_list) * quantile)])
    statistics([len(val) for _, val in confusionset.items()])

    chinese_list = zi_list[int(len(zi_list) * 0.4) : ]
    for key in confusionset:
        confusionset[key] = [val for val in confusionset[key] if val in chinese_list]
    new_confusion = {}
    for key in confusionset:
        if key in chinese_list:
            new_confusion[key] = confusionset[key]
    print('fix len',len(new_confusion))
    confusionset = new_confusion
    statistics([len(val) for _, val in confusionset.items()])
    return confusionset


confusionset = get_confusionset()



5615
fix len 5615
avg min/max 119.05574354407837 1 413
quantile 0.0 1
quantile 0.1 27
quantile 0.2 60
quantile 0.30000000000000004 81
quantile 0.4 96
quantile 0.5 114
quantile 0.6000000000000001 129
quantile 0.7000000000000001 151
quantile 0.8 177
quantile 0.9 215
fix len 4321
avg min/max 112.57556121268225 0 366
quantile 0.0 0
quantile 0.1 46
quantile 0.2 65
quantile 0.30000000000000004 78
quantile 0.4 91
quantile 0.5 104
quantile 0.6000000000000001 119
quantile 0.7000000000000001 138
quantile 0.8 160
quantile 0.9 192


In [2]:
from pypinyin import pinyin, Style
sentence = '我很快乐，happy，喜欢音乐,参差不齐，人参好吃'
pinyin_list = pinyin(sentence, style=Style.TONE3, heteronym=True, errors=lambda x: [['not chinese'] for _ in x])
print(pinyin_list)

[['wo3'], ['hen3'], ['kuai4'], ['le4'], ['not chinese'], ['not chinese'], ['not chinese'], ['not chinese'], ['not chinese'], ['not chinese'], ['not chinese'], ['xi3', 'xi1', 'chi4'], ['huan1'], ['yin1'], ['yue4'], ['not chinese'], ['cen1'], ['ci1'], ['bu4'], ['qi2'], ['not chinese'], ['ren2'], ['shen1'], ['hao3'], ['chi1']]


In [7]:
from tokenizers import BertWordPieceTokenizer
tokenizer = BertWordPieceTokenizer(
        '/home/ljh/model/ChineseBERT-base/vocab.txt', lowercase = True)
output = tokenizer.decode([101, 19476, 2861, 5018, 2697, 2418, 2137, 2526, 2989, 6835, 7390, 3198, 7313, 1359, 1265, 4638, 1770, 1767, 2582, 3416, 4495, 7357, 8020, 2697, 2418, 1139, 8021, 4510, 6605, 511, 3692, 4828, 2697, 2418, 3221, 6387, 1914, 1355, 4510, 3322, 4638, 6817, 868, 1333, 4415, 511, 891, 1963, 8024, 4572, 1779, 3181, 6760, 4638, 3340, 2501, 4828, 7188, 833, 772, 4495, 3198, 5753, 19789, 1767, 8024, 6821, 1348, 833, 4495, 2768, 4510, 1767, 8024, 886, 1765, 6943, 6818, 4638, 7308, 1726, 1750, 9459, 5445, 2697, 2418, 1139, 7441, 3837, 511, 102])
print(type(output),len(output),len([101, 19476, 2861, 5018, 2697, 2418, 2137, 2526, 2989, 6835, 7390, 3198, 7313, 1359, 1265, 4638, 1770, 1767, 2582, 3416, 4495, 7357, 8020, 2697, 2418, 1139, 8021, 4510, 6605, 511, 3692, 4828, 2697, 2418, 3221, 6387, 1914, 1355, 4510, 3322, 4638, 6817, 868, 1333, 4415, 511, 891, 1963, 8024, 4572, 1779, 3181, 6760, 4638, 3340, 2501, 4828, 7188, 833, 772, 4495, 3198, 5753, 19789, 1767, 8024, 6821, 1348, 833, 4495, 2768, 4510, 1767, 8024, 886, 1765, 6943, 6818, 4638, 7308, 1726, 1750, 9459, 5445, 2697, 2418, 1139, 7441, 3837, 511, 102]))


<class 'str'> 181 91


In [7]:
import torch
import json
file = '/home/ljh/CSC/Enhanced_Syllable_Feature/data/finetun_data/train_all'
with open(file, 'r' ,encoding='utf8') as f:
    data = list(f.readlines())
num = num2 = 0
for ex in data:
    example = json.loads(ex)
    input_ids, pinyin_ids, label,pinyin_label = example['input_ids'], example['pinyin_ids'], example['label'], example['pinyin_label']
    example_id,src,tokens_size = example['id'], example['src'], example['tokens_size']
    # convert list to tensor
    input_ids = torch.LongTensor(input_ids)
    pinyin_ids = torch.LongTensor(pinyin_ids).view(-1)

    label = torch.LongTensor(label)
    pinyin_label=torch.LongTensor(pinyin_label)
    assert input_ids.shape == label.shape
    assert input_ids.shape[0] * 8 == pinyin_ids.shape[0]
    assert input_ids.shape[0] == pinyin_label.shape[0]
    assert input_ids.shape[0] < 512
    if input_ids.shape[0]> 256:
        num +=1
    if input_ids.shape[0]> 192:
        num2 +=1
print(num, num2)
    

1 2
